In [1]:
import cv2
import numpy as np
import time
from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage.feature import peak_local_max

In [2]:
def contrast_strech(image):
    a = 0
    b = 255
    c = image.min()
    d_max = image.max()
    image_new = image
    image_new[:,:,:] = (image[:,:,:] - c)* ((b-a)/(d_max - c)) + a #(I-c)(b-a/max - min) + a
    return image_new

In [3]:
def convolve_max_filtered(image,N):
    h,w = image.shape
    output = np.zeros((h,w),dtype = np.uint8)
    #filter_new = np.array([[1]*N]*N  ,dtype ="int")  ## N*N filter with 1 in all places
    #need to pad the image according to filter size
    p = int((N-1)/2) #paading of zeros required
    padded_picture = cv2.copyMakeBorder(image.copy(),p,p,p,p,cv2.BORDER_CONSTANT,value = 0)
    for y in np.arange(p,h+p):
        for x in np.arange(p,w+p):
                roi = padded_picture[y-p:y+p+1, x-p:x+p+1]
                max_pixel = np.amax((roi)) #directly finding the max value from surrounding pixels and replacing it
                output[y-p,x-p]= max_pixel
    output = (output*255).astype("uint8")
    return output

In [4]:
def convolve_min_filtered(image,N):
    h,w = image.shape
    output = np.full((h,w),255,dtype = np.uint8)
    #filter_new = np.array([[1]*N]*N  ,dtype ="int")  ## N*N filter with 1 in all places
    #need to pad the image according to filter size
    p = int((N-1)/2) #paading of zeros required
    padded_picture = cv2.copyMakeBorder(image.copy(),p,p,p,p,cv2.BORDER_CONSTANT,value = 255)
    
    for y in np.arange(p,h+p):
        for x in np.arange(p,w+p):
                roi = padded_picture[y-p:y+p+1, x-p:x+p+1]
                min_pixel = np.amin((roi)) #directly finding the max value from surrounding pixels and replacing it
                output[y-p,x-p]= min_pixel
           
    output = (output*255).astype("uint8")
    return output

In [5]:
def remove_background(image,back_img):
    h,w = image.shape
    output = np.zeros((h,w),dtype = "uint8")
    output = np.subtract(image,back_img)
    return output

In [6]:
def run_algo(image,filter_size, M):
    if M == 0:
        max_filter =  convolve_max_filtered(image,filter_size)
        #min_filter =  convolve_min_filtered(max_filter,filter_size)
        final_image = remove_background(image, max_filter)
    else:
        min_filter = convolve_min_filtered(image,filter_size)
        max_filter = convolve_max_filtered(min_filter,filter_size)
        final_image = remove_background(image, max_filter)
    return final_image

In [7]:
def apply_watershed(img):
    #img_array = np.array(img.convert('L'))
    img_array = img.copy()
    distance = ndi.distance_transform_edt(img_array)
    markers = ndi.label(peak_local_max(distance, indices = False, labels = img_array))[0]
    ws_labels = watershed(-distance, markers, mask = img_array)
    return ws_labels

In [8]:
def image_sharpening(image):
    sigma = 10
    a = 10
    blur_image = cv2.GaussianBlur(image,(0,0),sigma)  #L = I*G
    #H = image - blur_image #H = I - L
    H = cv2.subtract(image,blur_image)
    final_image = H*a + image #o = I+H*a
    return final_image

In [9]:
cap = cv2.VideoCapture("C:/Users/Deepansh/Desktop/COMP 9517/GROUP PROJECT/images/DIC-C2DH-HeLa/Sequence 2/t%03d.tif")
filter_size = 3
M = 0
while(1):
    ret, frame = cap.read()
    if (ret) :  
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = run_algo(gray, filter_size,M)
        frame = (255-frame)
        #frame = image_sharpening(frame)
        #frame = apply_watershed(frame)
        cv2.imshow('image',frame)
        time.sleep(0.5)
    else:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    # 'q' to quit playback
    if (cv2.waitKey(1) & 0xFF == ord('q')):
        break
        
cap.release()
cv2.destroyAllWindows()